In [ ]:
import os 
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

data_path_2024 = os.getcwd()+'/data/final_2024_player.csv'
data_path_2025 = os.getcwd()+'/data/final_2025_player.csv'

df_2024 = pd.read_csv(data_path_2024)
df_2025 = pd.read_csv(data_path_2025)

df = pd.concat([df_2024, df_2025])

df.head()

In [ ]:
df.isnull().sum()

In [ ]:
df.describe()

### NBA Salary Distribution

In [ ]:
bins = np.arange(0, df['Next_Year_Salary'].max()+5000000, 5000000)

plt.hist(df['Next_Year_Salary'], bins = bins, edgecolor = 'black')
ax = plt.gca()

ax.xaxis.set_major_formatter(
    plt.FuncFormatter(lambda x, _: f"${x/1e6:.0f}M")
)

plt.xlabel('Next Year Salary')
plt.ylabel('Frequency')
plt.title('Distribution of NBA Player Salaries')

plt.savefig("outputs/salary_distribution.png")

The distribution of NBA salaries is clearly right skewed. Most players earn below 5 million dollars. This makes sense, because there are very few "superstars" who are deserving of a salary above 30 million, and majority of NBA players are role players or depth pieces that aren't expected to make much relative to others in their careers. 

### Salary and Stats

In [ ]:
plt.scatter(df['BPM'], df['Next_Year_Salary'])
ax = plt.gca()

ax.yaxis.set_major_formatter(
    plt.FuncFormatter(lambda x, _: f"${x/1e6:.0f}M")
)
plt.xlabel("Box Plus Minus")
plt.ylabel("Next Year Salary")
plt.title("Box Plus Minus Vs. Salary")

Box plus minus is a popular basketball advanced statistic that estimates a player's contribution to the team's performance, both offensively and defensively. A higher box plus minus score means that a player is contributing more towards their team's  success. It seems like there is a general positive trend in that players with higher BPM scores can expect to see higher salaries. However, there are a couple outliers, so we'll subset the dataset to only include players with BPM scores higher than -10 and replot this relationship. 

In [ ]:
subset = df[df['BPM'] > -10]

plt.scatter(subset['BPM'], subset['Next_Year_Salary'])
ax = plt.gca()

ax.yaxis.set_major_formatter(
    plt.FuncFormatter(lambda x, _: f"${x/1e6:.0f}M")
)
plt.xlabel("Box Plus Minus")
plt.ylabel("Next Year Salary")
plt.title("Box Plus Minus Vs. Salary")

In [ ]:
print(f"Correlation coefficient: {df['BPM'].corr(df['Next_Year_Salary'])}")

print(f"Subsetted Correlation coefficient: {subset['BPM'].corr(subset['Next_Year_Salary'])}")


After removing the outliers, the scatterplot gives a better view of the relationship between BPM and NBA salaries. The correlation coefficient confirms that there is a moderately strong positive association between the two.  

Let's do the same with VORP now

In [ ]:
plt.scatter(df['VORP'], df['Next_Year_Salary'])
ax = plt.gca()

ax.yaxis.set_major_formatter(
    plt.FuncFormatter(lambda x, _: f"${x/1e6:.0f}M")
)
plt.xlabel("VORP Minus")
plt.ylabel("Next Year Salary")
plt.title("VORP Vs. Salary")

In [ ]:
print(f"Correlation coefficient: {df['VORP'].corr(df['Next_Year_Salary'])}")


VORP is another popular advanced stat that estimates a player's overall contribution to a team compared to an average NBA player. It shows the same positive association with NBA salaries as BPM did; however, the correlation coefficient does show a stronger linear relationship than with BPM.

I'll combine both plots into one figure

In [ ]:
fig, axs = plt.subplots(nrows = 1, ncols = 2, figsize = (10,6))

plt.subplots_adjust(wspace=0.35)

axs[0].scatter(subset['BPM'], subset['Next_Year_Salary'])
axs[1].scatter(df['VORP'], df['Next_Year_Salary'])

axs[0].set_xlabel("Box Plus Minus (BPM)")
axs[0].set_ylabel("Salary")
axs[0].set_title("Box Plus Minus Vs. Salary")

axs[1].set_xlabel("Value Over Replacement Player (VORP)")
axs[1].set_ylabel("Salary")
axs[1].set_title("VORP Vs. Salary")

formatter = plt.FuncFormatter(lambda y, _: f"${y/1e6:.0f}M")

for ax in axs:
    ax.yaxis.set_major_formatter(formatter)

plt.savefig("outputs/bpm_vorp_vs_salary.png")

Let's take a look at how offensive and defensive metrics relate to NBA salaries

In [ ]:
good_scoring = df[(df['FGA'] >= 10) & (df['FG%'] >= 0.45)]
bad_scoring = df[~(df['FGA'] >= 10) & (df['FG%'] >= 0.45)]

plt.boxplot([good_scoring['Next_Year_Salary'], bad_scoring['Next_Year_Salary']], labels = ['Prolific Scorers', 'Non Prolific Scorers'])

ax = plt.gca()

ax.yaxis.set_major_formatter(
    plt.FuncFormatter(lambda x, _: f"${x/1e6:.0f}M")
)

plt.ylabel('Salary')
plt.title('Salary Comparison Between Prolific Scoring Players and Non Prolific Scoring Players')

plt.savefig("outputs/scoring_volume_salary_comparison.png")

This plot shows that there is quite a big difference in salaries between efficient scorers in the league and those who aren't. Moreover, the boxplot for prolific scorers is about normally distributed, while the boxplot for nonprolific scorers is definitely right skewed. This shows that most players who aren't efficiently scoring for their teams earn significantly less than those that do. This is probably a big factor that goes into determining a player's salary, which makes sense, as basketball is primarily an offensively minded sport. 

In [ ]:
plt.scatter(df['DBPM'], df['Next_Year_Salary'])
ax = plt.gca()

ax.yaxis.set_major_formatter(
    plt.FuncFormatter(lambda x, _: f"${x/1e6:.0f}M")
)
plt.xlabel("Defensive Box Plus Minus")
plt.ylabel("Next Year Salary")
plt.title("Defensive Box Plus Minus Vs. Salary")

plt.savefig("outputs/dbpm_vs_salary.png")

In [ ]:
print(f"Correlation coefficient: {df['DBPM'].corr(df['Next_Year_Salary'])}")


It's difficult to group players into defensively good and bad categories using defensive stats, as they aren't as accurate as offensive stats in evaluating a player's effectiveness. Instead, we used defensive box plus minus, which is essentially the same as box plus minus, but it evaluates a player's defensive efficiency/potential. The scatterplot and correlation coefficient both show that there isn't much of a relationship between defensive BPM and salaries. 

The last two plots show that offensive potential is more valued than defensive potential in evaluating a player's worth to a team through their salary. There is a clear salary difference between good offensive players and more average players, while there is not much of a relationship between defensive stats and NBA salaries. 

In [ ]:
fig, axs = plt.subplots(nrows = 1, ncols = 2, figsize = (12, 6))

axs[0].scatter(df['OBPM'], df['BPM'])
axs[1].scatter(df['DBPM'], df['BPM'])

axs[0].set_xlabel("Offensive Box Plus Minus")
axs[0].set_ylabel("Box Plus Minus")

axs[1].set_xlabel("Defensive Box Plus Minus")
axs[1].set_ylabel("Box Plus Minus")

axs[0].set_title("OBPM Vs. BPM")
axs[1].set_title("DBPM Vs. BPM")

plt.savefig("outputs/obpm_dbpm_vs_bpm.png")

Overall BPM is the sum of Offensive BPM and Defensive BPM. Our earlier analysis shows that there is a higher correlation between offensive stats and salaries than for defensive stats. We also showed a general positive association between BPM and salaries. Out of curiosity, I wanted to see the relationship between OBPM/DBPM and general BPM. There is clearly a stronger positive relationship between OBPM and BPM than for DBPM, which aligns with our earlier analysis between stats and salaries. 

From a salary standpoint, it seems like an offensive superstar is worth more than a defensive superstar. 

### Salary In Relation to Experience and Awards 

In [ ]:
age_salary = df.groupby('Age')['Next_Year_Salary'].mean()

ax = plt.gca()

ax.yaxis.set_major_formatter(
    plt.FuncFormatter(lambda x, _: f"${x/1e6:.0f}M")
)

plt.bar(age_salary.index, age_salary.values)
plt.xlabel("Age")
plt.ylabel("Avg Salary")
plt.title("Average Salary by Age")
plt.show()

In [ ]:
df[df['Age'] == 40]

In [ ]:
subset = df[df['Age'] < 39]
age_salary = subset.groupby('Age')['Next_Year_Salary'].mean()

ax = plt.gca()

ax.yaxis.set_major_formatter(
    plt.FuncFormatter(lambda x, _: f"${x/1e6:.0f}M")
)

plt.bar(age_salary.index, age_salary.values)
plt.xlabel("Age")
plt.ylabel("Avg Salary")
plt.title("Average Salary by Age")
plt.show()

plt.savefig("outputs/avg_salary_by_age.png")

The 40 years old column stands out, but after inspecting the dataframe, it makes sense, as there is only one 40 year old player, which is Lebron James. Arguably the greatest player of all time, he has a salary of about $48 million. Other than that though, the graph looks to be bimodal, with peaks around 29 years and 34 years. This makes sense, as players in their late 20s are typically around their peaks, which would coincide with higher salaries. The second peak is around older players, but the only players to remain in the leage in that age range are typically older superstars whose careers are defined by longevity. It then falls off, as very old players in their late 30s aren't as efficient as they used to be, which coincides with smaller salaries. 

In [ ]:
awards_salary = df.groupby('NumOfAwards')['Next_Year_Salary'].mean()

plt.bar(awards_salary.index, awards_salary.values)

ax = plt.gca()

ax.yaxis.set_major_formatter(
    plt.FuncFormatter(lambda x, _: f"${x/1e6:.0f}M")
)

plt.xlabel('Number of Awards')
plt.ylabel("Average Salary")

plt.title("Average Salary By Number of Awards")

plt.savefig("outputs/avg_salary_by_awards.png")

In [ ]:
all_star = df[df['All-Star'] == 1]
non_all_star = df[df['All-Star'] == 0]

plt.boxplot([all_star['Next_Year_Salary'], non_all_star['Next_Year_Salary']], labels = ["All Star", "Non All Star"])
ax = plt.gca()

ax.yaxis.set_major_formatter(
    plt.FuncFormatter(lambda x, _: f"${x/1e6:.0f}M")
)

plt.ylabel("Salary")
plt.title("NBA Salary By All Star Selection")

plt.savefig("outputs/all_star_salary_comp.png")

There are many awards in the NBA most notably: Most Valuable Player (MVP), Defensive Player of the Year (DPOY), and All NBA team selections. As expected, players with more awards tend to have higher salaries, as those with more awards are naturally better players. Another important accolade is selection for the All Star team. The side by side boxplot shows that All Star Players do earn much more than those who aren't. However, there are a lot of non All Star players as outliers who make about the same as All Star players. This is because All Star nominations are extremely selective, as there is only a certain amount of players each year who can be selected. As a result, there are many extremely talented players who are left off of the All Star roster. 

### Salary By Position

In [ ]:
position_salary = df.groupby('Pos')['Next_Year_Salary'].mean()

plt.bar(position_salary.index, position_salary.values)

ax = plt.gca()

ax.yaxis.set_major_formatter(
    plt.FuncFormatter(lambda x, _: f"${x/1e6:.0f}M")
)

plt.xlabel("Position")
plt.ylabel("Average Salary")

plt.title("Average Salary By Position")

plt.savefig("outputs/salary_by_pos.png")

It seems like point guards on average are paid more than their peers. This makes sense, because usually the point guards are the players managing a team's offense. Oftentimes, team offenses are largely structured around the play of their pointguards. Due to their importance to teams, it does make sense that they are paid more on average than other positions. All other positions are on average paid around the same at around $12 million. 

In [ ]:
vif_columns = ['Age', 'G', 'GS', 'MP_x', 'FG', 'FGA',
       'FG%', '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA',
       'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'PER', 'TS%', '3PAr', 'FTr', 'ORB%', 'DRB%', 'TRB%', 'AST%',
       'STL%', 'BLK%', 'TOV%', 'USG%', 'OWS', 'DWS', 'WS', 'WS/48', 'OBPM',
       'DBPM', 'BPM', 'VORP', 'Experience', 'NumOfAwards',
       'All-Star', 'AwardWinner', 'FirstTeam', 'SecondTeam', 'ThirdTeam',
       'DefTeam1', 'DefTeam2']

non_numeric = ['Player', 'Team', 'Pos', 'Awards']

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.preprocessing import StandardScaler

df_vif = df[vif_columns].copy()
df_vif = df_vif.fillna(df_vif.median())

scaler = StandardScaler()
X_scaled = scaler.fit_transform(df_vif)

vif_df = pd.DataFrame()
vif_df['feature'] = df_vif.columns
vif_df['VIF'] = [variance_inflation_factor(X_scaled, i) 
                 for i in range(X_scaled.shape[1])]


In [ ]:
vif_df.sort_values('VIF', ascending = False)

A lot of the variables in our dataset are highly correlated, as they are functions of each other. For instance FG% = FG/FGA. Let's drop all of the counting stats and keep percentage based stats. We'll also drop most of the advanced stats but keep BPM, as most advanced stats are factored into BPM. 

In [ ]:
cols2drop = ['G', 'GS', 'FG', 'FGA', '3P', '3PA', '2P', '2PA', 'FT', 'FTA', 'ORB', 'DRB', 'TRB', '3PAr', 'FTr', 'STL', 'AST', 'BLK', 'TOV', 'OWS', 'DWS', 'WS', 'OBPM', 'DBPM', 'VORP', 'Experience', 'FG%', '3P%', '2P%', 'ORB%', 'DRB%', 'eFG%', 'WS/48', 'PER', 'PTS']

df_vif.drop(cols2drop, axis=1, inplace=True)

df_vif.columns

In [ ]:
df_vif = df_vif.fillna(df_vif.median())

scaler = StandardScaler()
X_scaled = scaler.fit_transform(df_vif)

vif_df = pd.DataFrame()
vif_df['feature'] = df_vif.columns
vif_df['VIF'] = [variance_inflation_factor(X_scaled, i) 
                 for i in range(X_scaled.shape[1])]


In [ ]:
vif_df.sort_values('VIF', ascending = False)

Much better, we can keep all of these columns, as the VIF values are all less than 10. 

In [ ]:
cols2keep = list(vif_df.feature)

In [ ]:
cols2keep

In [ ]:
final_df = df[cols2keep+['Pos', 'Next_Year_Salary']]

final_df.columns